In [1]:
!pip install numpy pandas geopandas matplotlib networkx osmnx contextily folium shapely EE
import networkx as nx # need networkx >= 2.5
import osmnx as ox
import contextily as ctx
from matplotlib import pyplot as plt
from shapely.geometry import Polygon, Point
import folium
import numpy as np
import matplotlib.patches as mpatches
import geopandas as gpd
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import pandas as pd

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 51.7 MB/s eta 0:00:00
  Created wheel for EE: filename=ee-0.2-py3-none-any.whl size=3655 sha256=e8ab455af789c94a1acecca9717cd1a8887090cb06cf0dc54e81dbbc324f6a25
  Stored in directory: /root/.cache/pip/wheels/a0/6d/ee/1d25db5c1257f6b86266fa4c20a2439110c42e8759486066f7
Successfully built EE


In [ ]:
hospital_df = gpd.read_file('Hospitals_2025.geojson')
shelters_df = gpd.read_file('Shelters_2025.geojson')
game_grid_df = gpd.read_file('game_grid_2025.geojson')

In [ ]:
game_grid_df['coords'] = game_grid_df.apply(lambda x: (x['EASTING'], x['NORTHING']), axis=1)
game_grid_df = game_grid_df.drop_duplicates('coords')
game_grid_df

In [ ]:
poly1 = game_grid_df["geometry"][0]
poly2 = game_grid_df["geometry"][1]

fig, ax = plt.subplots()
for poly, color in zip([poly1, poly2], ['blue', 'green']):
    x, y = poly.exterior.xy
    ax.fill(x, y, alpha=0.5, fc=color, ec='black')

ax.set_aspect('equal')
ax.set_title("Polygons from Data File")
plt.show()

In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry.base import BaseGeometry

G = nx.Graph()
sindex = game_grid_df.sindex

# for idx, row in game_grid_df.iterrows():
#     G.add_node(idx, transport_score=row['transport_score'], geometry=row.geometry)

for idx, row in game_grid_df.iterrows():
    geom = row.geometry
    mgrs = row['MGRS']

    possible_matches_idx = list(sindex.intersection(geom.bounds))
    for potential_idx in possible_matches_idx:
        if potential_idx <= idx:
            continue

        potential_row = game_grid_df.iloc[potential_idx]
        if geom.intersects(potential_row.geometry) and mgrs != potential_row['MGRS']:
            weight = 20.0 / (row['transport_score'] + potential_row['transport_score'])
            G.add_edge(idx, potential_idx, weight=weight)


In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())
# print(G.edges(5000))

In [ ]:
import networkx as nx

for node, data in G.nodes(data=True):
    if 'geometry' in data:
        data['geometry'] = data['geometry'].wkt


nx.write_graphml(G, "graph_pre.graphml")